In [1]:
from __future__ import annotations # https://stackoverflow.com/questions/61544854/from-future-import-annotations

import os
import time
from pprint import pprint
from typing import Literal

import asyncio
import nest_asyncio
from dotenv import load_dotenv

import matplotlib.pyplot as plt
from pydantic import BaseModel
from dataclasses import dataclass

from agents import (
    Agent, Runner, trace, set_default_openai_client,
    InputGuardrail, GuardrailFunctionOutput,
    WebSearchTool, ItemHelpers, TResponseInputItem
)
from agents.exceptions import InputGuardrailTripwireTriggered

from openai import AsyncOpenAI

In [ ]:
# Use env var for API key and set a long timeout
client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"), timeout=600.0)
set_default_openai_client(client)
# os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "1" # Disable tracing for Zero Data Retention (ZDR) Organizations

In [2]:
nest_asyncio.apply()

In [3]:
load_dotenv(override=True)

True

# 1.0 Introduction

### 1.1 Basic Concepts

To get started, you'll need to be aware of 3 very important [primitives]():

> **Agents**, which are LLMs equipped with instructions and tools.  
> **Handoffs**, which allow agents to delegate to other agents for specific tasks.  
> **Guardrails**, which enable the inputs to agents to be validated.  

With that in mind, we'll generally do 3 steps when calling our Agents:
1. Creating an `Agent` instance.
2. Using `with trace()` to track the Agent.
3. Calling `runner.run()` to run the Agent.

Additionally:
- For tracking our Agent's interactions & output we can go to: https://platform.openai.com/traces
- For documentation, we can go to: https://openai.github.io/openai-agents-python/
- Additional tutorials: https://openai.github.io/openai-agents-python/examples/

In [3]:
agent = Agent(
    name="Jokester",
    instructions="You are a comedian. You are going to tell jokes.",
    model="gpt-4o-mini",
)

In [7]:
with trace("Testing-2025-08-23"):
    result = await Runner.run(agent, "Tell me a joke about Ronaldo vs. Messi.")
    print(result.final_output)

Why don’t Ronaldo and Messi ever play hide and seek?

Because good luck hiding when Messi can dribble past you, and Ronaldo will just take a selfie from wherever he is!


In [8]:
result

RunResult(input='Tell me a joke about Ronaldo vs. Messi.', new_items=[MessageOutputItem(agent=Agent(name='Jokester', instructions='You are a comedian. You are going to tell jokes.', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_68a9c87aa8e0819f809b3e4988ec771e0b13905296926762', content=[ResponseOutputText(annotations=[], text='Why don’t Ronaldo and Messi ever play hide and seek?\n\nBecause good luck hiding when Messi can dribble past you, and Ronaldo will just take 

In [9]:
print(result)

RunResult:
- Last agent: Agent(name="Jokester", ...)
- Final output (str):
    Why don’t Ronaldo and Messi ever play hide and seek?
    
    Because good luck hiding when Messi can dribble past you, and Ronaldo will just take a selfie from wherever he is!
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


### 1.2 Using multiple agents with guardrails

In [12]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

In [13]:
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

In [14]:
math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

In [15]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

In [23]:
pprint(history_tutor_agent)

Agent(name='History Tutor',
      instructions='You provide assistance with historical queries. Explain '
                   'important events and context clearly.',
      handoff_description='Specialist agent for historical questions',
      handoffs=[],
      model=None,
      model_settings=ModelSettings(temperature=None,
                                   top_p=None,
                                   frequency_penalty=None,
                                   presence_penalty=None,
                                   tool_choice=None,
                                   parallel_tool_calls=None,
                                   truncation=None,
                                   max_tokens=None,
                                   reasoning=None,
                                   metadata=None,
                                   store=None,
                                   include_usage=None,
                                   extra_query=None,
                                   

In [16]:
async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

In [17]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

In [18]:
async def main():
    # Example 1: History question
    try:
        result = await Runner.run(triage_agent, "who was the first president of the united states?")
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

    # Example 2: General/philosophical question
    try:
        result = await Runner.run(triage_agent, "What is the meaning of life?")
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

In [ ]:
# if we ran this via a Python script, use `asyncio.run(main())`
await main()

The first President of the United States was George Washington. He served from April 30, 1789, to March 4, 1797. Washington is known for leading the Continental Army to victory over the British in the American Revolutionary War and for presiding over the Constitutional Convention of 1787, which drafted the United States Constitution. His presidency set many precedents, including the formation of a Cabinet and the tradition of a two-term limit, which later became law with the 22nd Amendment. Washington is often referred to as the "Father of His Country" for his pivotal role in founding the United States.
Guardrail blocked this input: Guardrail InputGuardrail triggered tripwire


### 1.3 WebSearch

In [10]:
async def main():
    agent = Agent(
        name="Web searcher",
        instructions="You are a helpful agent who's an expert web results synthesizer.",
        tools=[WebSearchTool(user_location={"type": "approximate", "city": "Kuala Lumpur"})],
    )

    with trace("Web search example"):
        result = await Runner.run(
            agent,
            "search the web for 'local sports news' and give me 1 interesting update in a sentence.",
        )
        pprint(result)
        print("")
        print(result.final_output)

In [11]:
asyncio.run(main())

RunResult(input="search the web for 'local sports news' and give me 1 "
                'interesting update in a sentence.',
          new_items=[ToolCallItem(agent=Agent(name='Web searcher',
                                              handoff_description=None,
                                              tools=[WebSearchTool(user_location={'city': 'Kuala '
                                                                                          'Lumpur',
                                                                                  'type': 'approximate'},
                                                                   search_context_size='medium')],
                                              mcp_servers=[],
                                              mcp_config={},
                                              instructions='You are a helpful '
                                                           "agent who's an "
                                                    

### 1.4 LLM-as-a-Judge

In [14]:
story_outline_generator = Agent(
    name="story_outline_generator",
    instructions=(
        "You generate a very short story outline based on the user's input. "
        "If there is any feedback provided, use it to improve the outline."
    ),
)

In [15]:
@dataclass
class EvaluationFeedback:
    feedback: str
    score: Literal["pass", "needs_improvement", "fail"]

In [ ]:
Agent[None]

agents.agent.Agent

In [18]:
evaluator = Agent[None](
    name="evaluator",
    instructions=(
        "You evaluate a story outline and decide if it's good enough. "
        "If it's not good enough, you provide feedback on what needs to be improved. "
        "Never give it a pass on the first try. After 5 attempts, you can give it a pass if the story outline is good enough - do not go for perfection"
    ),
    output_type=EvaluationFeedback,
)

In [19]:
async def main() -> None:
    msg = input("What kind of story would you like to hear? ")
    input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]
    latest_outline: str | None = None

    pprint(input_items)
    print("")

    # We'll run the entire workflow in a single trace
    with trace("LLM as a judge"):
        while True:
            story_outline_result = await Runner.run(
                story_outline_generator,
                input_items,
            )

            input_items = story_outline_result.to_input_list()
            pprint(input_items)
            print("")
            latest_outline = ItemHelpers.text_message_outputs(story_outline_result.new_items)
            print("Story outline generated")

            evaluator_result = await Runner.run(evaluator, input_items)
            result: EvaluationFeedback = evaluator_result.final_output

            print(f"Evaluator score: {result.score}")

            if result.score == "pass":
                print("Story outline is good enough, exiting.")
                break

            print("Re-running with feedback")

            input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})

    print(f"Final story outline: {latest_outline}")

In [20]:
asyncio.run(main())

[{'content': 'About Taylor Swift finally finding the love of her life. :)',
  'role': 'user'}]

[{'content': 'About Taylor Swift finally finding the love of her life. :)',
  'role': 'user'},
 {'content': [{'annotations': [],
               'logprobs': [],
               'text': '**Title:** "Love’s Encore"\n'
                       '\n'
                       '**Outline:**  \n'
                       'Taylor Swift, after years in the global spotlight and '
                       'facing heartbreaks that inspired her greatest hits, '
                       'unexpectedly connects with a humble, compassionate '
                       'music teacher from her hometown. As they bond over '
                       'their shared passion for songwriting and simple joys, '
                       'Taylor learns to trust love again. Together, they '
                       'navigate fame, privacy, and their own vulnerabilities, '
                       'discovering that true love often arrives when y

### 1.5 Basic Research Agent

In [4]:
# Define the research agent
research_agent = Agent(
    name="Research Agent",
    model="o4-mini-deep-research-2025-06-26",
    tools=[WebSearchTool()],
    instructions="You perform deep empirical research based on the user's question."
)

In [10]:
# Async function to run the research and print streaming progress
async def basic_research(query):
    print(f"Researching: {query}")
    result_stream = Runner.run_streamed(
        research_agent,
        query
    )

    async for ev in result_stream.stream_events():
        print(f"Event Type: {ev.type}")
        print(f"Event Data: {ev}")
        if ev.type == "agent_updated_stream_event":
            print(f"\n--- switched to agent: {ev.new_agent.name} ---")
            print(f"\n--- RESEARCHING ---")
        elif (
            ev.type == "raw_response_event"
            and hasattr(ev.data, "item")
            and hasattr(ev.data.item, "action")
        ):
            action = ev.data.item.action or {}
            if action.get("type") == "search":
                print(f"[Web search] query={action.get('query')!r}")
    # streaming is complete → final_output is now populated
    return result_stream.final_output

In [11]:
# Run the research and print the result
result = await basic_research("Recommend me some places to go to while in Georgetown, Penang, Malaysia. I'll be near the Earnest Hotel.")
print(result)

Researching: Recommend me some places to go to while in Georgetown, Penang, Malaysia. I'll be near the Earnest Hotel.
Event Type: agent_updated_stream_event
Event Data: AgentUpdatedStreamEvent(new_agent=Agent(name='Research Agent', handoff_description=None, tools=[WebSearchTool(user_location=None, search_context_size='medium')], mcp_servers=[], mcp_config={}, instructions="You perform deep empirical research based on the user's question.", prompt=None, handoffs=[], model='o4-mini-deep-research-2025-06-26', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', re

AttributeError: 'ActionSearch' object has no attribute 'get'

In [18]:
type(action)

openai.types.responses.response_function_web_search.ActionSearch

In [26]:
query = "Recommend me some places to go to while in Georgetown, Penang, Malaysia. I'll be near the Earnest Hotel."
print(f"Researching: {query}")
result_stream = Runner.run_streamed(
    research_agent,
    query
)

async for ev in result_stream.stream_events():
    print(f"Event Type: {ev.type}")
    print(f"Event Data: {ev}")
    if ev.type == "agent_updated_stream_event":
        print(f"\n--- switched to agent: {ev.new_agent.name} ---")
        print(f"\n--- RESEARCHING ---")
    elif (
        ev.type == "raw_response_event"
        and hasattr(ev.data, "item")
        and hasattr(ev.data.item, "action")
    ):
        action = ev.data.item.action or {}
        if dict(action).get("type") == "search":
            print(f"[Web search] query={dict(action).get('query')!r}")

# streaming is complete → final_output is now populated
print(result_stream.final_output)

Researching: Recommend me some places to go to while in Georgetown, Penang, Malaysia. I'll be near the Earnest Hotel.
Event Type: agent_updated_stream_event
Event Data: AgentUpdatedStreamEvent(new_agent=Agent(name='Research Agent', handoff_description=None, tools=[WebSearchTool(user_location=None, search_context_size='medium')], mcp_servers=[], mcp_config={}, instructions="You perform deep empirical research based on the user's question.", prompt=None, handoffs=[], model='o4-mini-deep-research-2025-06-26', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', re

In [30]:
pprint(ev)

RunItemStreamEvent(name='message_output_created',
                   item=MessageOutputItem(agent=Agent(name='Research Agent',
                                                      handoff_description=None,
                                                      tools=[WebSearchTool(user_location=None,
                                                                           search_context_size='medium')],
                                                      mcp_servers=[],
                                                      mcp_config={},
                                                      instructions='You '
                                                                   'perform '
                                                                   'deep '
                                                                   'empirical '
                                                                   'research '
                                                                  

In [31]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    email: str | None = None  # Optional field with a default of None

# Valid data
user_data = {"id": 1, "name": "Alice"}
user = User(**user_data)
print(user)

id=1 name='Alice' email=None


In [36]:
user.name

'Alice'

In [35]:
user.get('id')

AttributeError: 'User' object has no attribute 'get'

In [40]:
x = {}

testing = x.get("type")
print(testing)

assert testing == None, "testing can't be None"

None
